In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import glob
import pydicom
import matplotlib.pyplot as plt
from time import time_ns


In [ ]:
#model = train()

In [4]:
import torch
model = torch.load("models/my_unet_model_.pt")

In [5]:
def load_test_dataset():
    train_images = os.listdir(f'{data_location}/test_images')
    train_images = list(filter(lambda x: x.find('.DS') == -1, train_images))
    train_images = [(x, f'{data_location}/test_images/{x}') for x in
                    train_images]

    image_metadata_set = {p[0]: {'folder_path': p[1],
                                 'SeriesInstanceUIDs': []
                                 }
                          for p in train_images}

    for m in image_metadata_set:
        image_metadata_set[m]['SeriesInstanceUIDs'] = list(
            filter(lambda x: x.find('.DS') == -1,
                   os.listdir(image_metadata_set[m]['folder_path'])
                   )
        )

    df_meta_f = pd.read_csv(f'{data_location}//test_series_descriptions.csv')

    for k in tqdm(image_metadata_set):
        for s in image_metadata_set[k]['SeriesInstanceUIDs']:
            if 'SeriesDescriptions' not in image_metadata_set[k]:
                image_metadata_set[k]['SeriesDescriptions'] = []
            try:
                image_metadata_set[k]['SeriesDescriptions'].append(
                    df_meta_f[(df_meta_f['study_id'] == int(k)) &
                              (df_meta_f['series_id'] == int(s))]['series_description'].iloc[0])
            except:
                print("Failed on", s, k)

    return image_metadata_set

In [6]:
data_location = "data/rsna-2024-lumbar-spine-degenerative-classification"
test_images = load_test_dataset()

100%|██████████| 1/1 [00:00<00:00, 296.73it/s]


In [7]:
def get_test_imagesets_for_patient(sample_patient, image_metadata_set):
    # !TODO: Index by some patient id
    ptobj = image_metadata_set[sample_patient]

    im_list_dcm = {}
    for idx, i in enumerate(ptobj['SeriesInstanceUIDs']):
        im_list_dcm[i] = {'images': [], 'description': ptobj['SeriesDescriptions'][idx]}
        images = glob.glob(f"{ptobj['folder_path']}/{ptobj['SeriesInstanceUIDs'][idx]}/*.dcm")
        for j in sorted(images, key=lambda x: int(x.split('/')[-1].replace('.dcm', ''))):
            im_list_dcm[i]['images'].append({
                'SOPInstanceUID': j.split('/')[-1].replace('.dcm', ''),
                'dicom': pydicom.dcmread(j)})

    return {e['description']: [x['dicom'].pixel_array for x in e['images']] for e in im_list_dcm.values()}


In [8]:
def yield_images(image_metadata_set, subjects):
    image_set = []
    corresponding_subject = []
    for subject in subjects:
        test_imageset = get_test_imagesets_for_patient(subject, image_metadata_set)
        image_set += [np.array(e, dtype=np.int32) for e in test_imageset["Axial T2"]]
        image_set += [np.array(e, dtype=np.int32) for e in test_imageset["Sagittal T2/STIR"]]
        image_set += [np.array(e, dtype=np.int32) for e in test_imageset["Sagittal T1"]]
        corresponding_subject.append(subject)
        corresponding_subject.append(subject)
        corresponding_subject.append(subject)

    return image_set, corresponding_subject


In [9]:
subjects = list(test_images.keys())
test_images, corresponding_subjects = yield_images(test_images, subjects)

In [10]:
model.eval()

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [11]:
import torchvision.transforms as transforms

tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])


In [12]:
tfd_input = torch.stack([tf(e).float() for e in test_images])

In [ ]:
out = model(tfd_input)

In [1]:
sample_submission = pd.read_csv(f"{data_location}/sample_submission.csv")

NameError: name 'pd' is not defined